In [ ]:
from astropy.cosmology import Planck15 as cosmo

import requests
import time
import numpy as np
import matplotlib.patheffects as path_effects
import os
import h5py
from matplotlib.patches import Rectangle

baseUrl = 'http://www.tng-project.org/api/'

def get(path, params=None, max_retries=5, backoff_factor=2):
    # make HTTP GET request to path
    headers = {"api-key":"API KEY"}

    attempt = 0
    while attempt < max_retries:
        try:
            r = requests.get(path, params=params, headers=headers)
            r.raise_for_status()

            if r.headers['content-type'] == 'application/json':
                return r.json()
            
            if 'content-disposition' in r.headers:
                filename = r.headers['content-disposition'].split("filename=")[1]
                with open(filename, 'wb') as f:
                    f.write(r.content)
                return filename
            return r  # fallback
        
        except Exception as e:
            attempt += 1
            wait = backoff_factor ** attempt
            print(f"[Retry {attempt}/{max_retries}] Request failed: {e}. Retrying in {wait}s...")
            time.sleep(wait)

    raise RuntimeError(f"Failed to GET {path} after {max_retries} retries.")

# Issue a request to the API root
r = get(baseUrl)

# Print out all the simulation names
names = [sim['name'] for sim in r['simulations']]
# Get the index of TNG300-1
i = names.index('TNG300-1')
# Get the info of simulation Illustris-3
sim = get( r['simulations'][i]['url'] )
sim.keys()

# get the snaps info this simulation
snaps = get(sim['snapshots'])

# Sim Box parameters
Snap_Index = 92 # the snapshots index in the total 100 snapshots taking at different z
BoxSize = sim['boxsize'] # unit: ckpc/h
Redshift = snaps[Snap_Index]['redshift'] # current redshift of our current snap

# Short description of cosmological parameters Planck2015
h_at0 = cosmo.h

LowerMass_lim = 10**3 * h_at0 # units 10^ 10 𝑀⊙/ℎ * h
HigherMass_lim = 10**8 * h_at0 # units 10^ 10 𝑀⊙/ℎ * h

In [ ]:
dm_mass= get('http://www.tng-project.org/api/TNG300-1/')['mass_dm']/h_at0 * 1e+10

In [ ]:
def get_mass_ratio(cat_name, halo_id):
    with h5py.File(cat_name) as cat:
        Halo_IDs = cat['Group/FOF_Halo_IDs'][:]
        GroupPos = cat['Group/GroupPos'][:]
        Group_R_Crit200 = cat['Group/Group_R_Crit200'][:]
        
        SubhaloGrNr = cat['Subhalo/SubhaloGrNr'][:]
        SubhaloMasses = cat['Subhalo/SubhaloMass'][:]
        SubhaloPos = cat['Subhalo/SubhaloPos'][:]

    halo_pos =  GroupPos[Halo_IDs == halo_id][0]
    halo_x, halo_y, halo_z = halo_pos[0], halo_pos[1], halo_pos[2]
    halo_r = Group_R_Crit200[Halo_IDs==halo_id]

    Subhalo_in = np.where(
        (SubhaloGrNr == halo_id) &
        (SubhaloPos[:,0] <= halo_x + halo_r) &
        (SubhaloPos[:,0] >= halo_x - halo_r) &
        (SubhaloPos[:,1] <= halo_y + halo_r) &
        (SubhaloPos[:,1] >= halo_y - halo_r) &
        (SubhaloPos[:,2] <= halo_z + 2*halo_r) &
        (SubhaloPos[:,2] >= halo_z - 2*halo_r) 
    )[0]

    SubhaloMass_in = SubhaloMasses[Subhalo_in]
    sorted_masses = np.sort(SubhaloMass_in)[::-1] 

    # get the most and second massive halo indices
    if len(sorted_masses) >= 2:
        m1, m2 = sorted_masses[0], sorted_masses[1]
        top_sorted = np.argsort(SubhaloMass_in)[::-1]
        top_indices = Subhalo_in[top_sorted[:2]] 
    elif len(sorted_masses) == 1:
        m1, m2 = sorted_masses[0], 0
        top_indices = [Subhalo_in[0]]
    else:
        m1, m2 = 0, 0
        top_indices = []
    
    # get positions
    top_positions = SubhaloPos[top_indices]  
    
    # convert to pixel pos
    relative_xy = top_positions[:, :2] - halo_pos[:2]
    pixel_coords = (relative_xy / (2*halo_r) * 240).astype(int) + 120  # center shift


    return m1, m2, m2 / m1 if m1 > 0 else np.nan, pixel_coords

In [ ]:
cat_name_300 = '/home/chuiyang/merger_trace/data/tng300/tng300_targetcat/TargetHalo_MergerCat_092.hdf5'

In [ ]:
m1, m2, ratio, pixel_coords = get_mass_ratio(cat_name_300, 142)

In [ ]:
with h5py.File(cat_name_300) as cat:
    Halo_IDs = cat['Group/FOF_Halo_IDs'][:]
    GroupPos = cat['Group/GroupPos'][:]
    Group_R_Crit200 = cat['Group/Group_R_Crit200'][:]
        
    SubhaloGrNr = cat['Subhalo/SubhaloGrNr'][:]
    SubhaloMasses = cat['Subhalo/SubhaloMass'][:]
    SubhaloPos = cat['Subhalo/SubhaloPos'][:]

halo_id = int(142)
halo_r200 = Group_R_Crit200[Halo_IDs==halo_id][0]

In [ ]:
z = 0.2
h = cosmo.h
a_atz = 1.0 / (1.0 + z)
pixel_length_ckpch = 2*halo_r200/240
pixel_length_kpc = pixel_length_ckpch * a_atz/h

length_kpc = 500
length_pixel = length_kpc/pixel_length_kpc

In [ ]:
pixel_length_kpc * 10

In [ ]:
import numpy as np
import h5py
from astropy.io import fits
import matplotlib.pyplot as plt

plt.style.use("seaborn-v0_8-whitegrid")
plt.rcParams.update({
    "font.size": 12,
    "font.family": "serif",
    "axes.labelsize": 13,
    "axes.titlesize": 13,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "legend.fontsize": 11,
    "pdf.fonttype": 42  # Ensures TrueType font embedded (for APJ)
})



def save_figure(fig, filename, dpi=300):
    """Save figure in both PDF (for APJ) and PNG (for local use) formats."""
    base = os.path.join("figures", filename)
    fig.savefig(f"{base}.pdf", bbox_inches='tight')  # APJ-compatible vector format
    #fig.savefig(f"{base}.png", dpi=dpi, bbox_inches='tight')  # For checking / internal use
    print(f"Saved: {base}.pdf and {base}.png")


# read aperture mass maps
fits_path = f'/home/chuiyang/merger_trace/data/tng300/tng300_aperture_mass_maps/sn_map_full_halo{halo_id}.fits'
with fits.open(fits_path) as hdul:
    sn_map = hdul[0].data

from scipy.ndimage import gaussian_filter

#sn_map_smoothed = gaussian_filter(sn_map, sigma=1.0)  
sn_map_smoothed = gaussian_filter(sn_map, sigma=10.0) 

# read particle snapshot
h5_path = f'/home/chuiyang/merger_trace/data/tng300/tng300_halos/snap_halo_{halo_id}.h5'
with h5py.File(h5_path, 'r') as f:
    
    dm_pos = f['PartType1/Coordinates'][:]  # shape: (N, 3)

    gas_pos = f['PartType0/Coordinates'][:]
    gas_mass = f['PartType0/Masses'][:]/h_at0 * 1e10


x1 = dm_pos[:, 0]
y1 = dm_pos[:, 1]
mass_dm_Mdot = np.ones(len(x1)) * dm_mass
surface_density_dm, xedges, yedges = np.histogram2d(
    x1, y1, bins=240, weights=mass_dm_Mdot
)

x0 = gas_pos[:, 0]
y0 = gas_pos[:, 1]
surface_density_gas, xedges, yedges = np.histogram2d(
    x0, y0, bins=240, weights=gas_mass
)
# convert to log scale
log_surf = np.log10(surface_density_dm+surface_density_gas + 1e6)  # avoid log(0)

# plot
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# subplot 1: aperture mass S/N map
im0 = axes[0].imshow(sn_map_smoothed, origin='lower', cmap='viridis')
axes[0].set_title('Aperture Mass S/N')
axes[0].set_xticks([]) 
axes[0].set_yticks([])
plt.colorbar(im0, ax=axes[0], fraction=0.046)

# subplot 2: surface density
im1 = axes[1].imshow(log_surf, origin='lower', cmap='inferno')
axes[1].set_xticks([])
axes[1].set_yticks([])  
axes[1].set_title('Surface Density (log $M_\\odot$ / pixel)')
plt.colorbar(im1, ax=axes[1], fraction=0.046)

labels = ['Central', 'Second']
colors = ['white', 'white']  
marker_facecolors = ['black', 'gray'] 
for i, (x, y) in enumerate(pixel_coords):
    axes[1].plot(x, y, 'o',
                 markerfacecolor=marker_facecolors[i],
                 markeredgecolor=colors[i],
                 markersize=6)
    axes[1].text(x+5, y+5, labels[i],
                 color='white',
                 fontsize=9,
                 weight='bold',
                 path_effects=[plt.matplotlib.patheffects.withStroke(linewidth=2, foreground='black')])

#fig.savefig("mock_aperture_mass_surface_density_halo142.png", dpi=300, bbox_inches='tight')
axes[0].grid(False)
axes[1].grid(False)


axes[0].add_patch(Rectangle((5, 5), length_pixel, 2, color='white'))
axes[0].text(11, 11, f'{length_kpc} kpc', color='white', fontsize=15, fontname='serif')

axes[1].add_patch(Rectangle((5, 5), length_pixel, 2, color='white'))
axes[1].text(11, 11, f'{length_kpc} kpc', color='white', fontsize=15, fontname='serif')

save_figure(plt.gcf(), "mock_aperture_mass_surface_density_halo142")
plt.show()


In [ ]:
halo_r200 * a_atz/h